In [1]:
#define document
#DKKD
Contents={}
Contents.update({"_Header":["Mã Số Doanh Nghiệp","Đăng ký lần đầu","Đăng ký thay đổi lần thứ"]})
Contents.update({"1.Tên công ty":["Tên Công ty viết bằng tiếng Việt","Tên công ty viết bằng tiếng nước ngoài","Tên công ty viết tắt"]})
Contents.update({"2.Địa chỉ trụ sở chính":["_Địa chỉ","Điện thoại","Fax","Email","Website"]})
Contents.update({"3.Vốn điều lệ":["Vốn điều lệ","Bằng Chữ","Mệnh giá cổ phần","Tổng số cổ phần"]})
Contents.update({"4.Người đại diện theo pháp luật của công ty":["Họ và tên","Giới tính","Chức danh","Sinh ngày","Dân tộc","Quốc tịch","Loại giấy tờ chứng thực cá nhân","Ngày cấp","Nơi cấp","Nơi đăng kí hộ khẩu thường trú","Chỗ ở hiện tại"]})
Contents.update({"_End":["TRƯỞNG PHÒNG"]}) # or {"_End":["None"]}

In [2]:
Keywords0=list(Contents.keys())
for x in Keywords0:
    Contents[x].append("_end")
Contents

{'_Header': ['Mã Số Doanh Nghiệp',
  'Đăng ký lần đầu',
  'Đăng ký thay đổi lần thứ',
  '_end'],
 '1.Tên công ty': ['Tên Công ty viết bằng tiếng Việt',
  'Tên công ty viết bằng tiếng nước ngoài',
  'Tên công ty viết tắt',
  '_end'],
 '2.Địa chỉ trụ sở chính': ['_Địa chỉ',
  'Điện thoại',
  'Fax',
  'Email',
  'Website',
  '_end'],
 '3.Vốn điều lệ': ['Vốn điều lệ',
  'Bằng Chữ',
  'Mệnh giá cổ phần',
  'Tổng số cổ phần',
  '_end'],
 '4.Người đại diện theo pháp luật của công ty': ['Họ và tên',
  'Giới tính',
  'Chức danh',
  'Sinh ngày',
  'Dân tộc',
  'Quốc tịch',
  'Loại giấy tờ chứng thực cá nhân',
  'Ngày cấp',
  'Nơi cấp',
  'Nơi đăng kí hộ khẩu thường trú',
  'Chỗ ở hiện tại',
  '_end'],
 '_End': ['TRƯỞNG PHÒNG', '_end']}

In [3]:
# clean STR
import re
import numpy as np
def clean_str(s):
    s=s.strip()
    s=re.sub('[*&/`~@#$%\^&*]+', '', s)
    return s.lower()

def compare_str(s1,s2):
    n1=len(s1)
    n2=len(s2)
    f=np.zeros((n1+1,n2+1))
    for i in range(n1+1):
        f[i][0]=1
    for j in range(n2+1):
        f[0][j]=1    
    f[0][0]=0
    if(s1[0]!=s2[0]):
        f[1][1]=1
    for i in range(0,n1):
        for j in range(0,n2):
            if(s1[i]==s2[j]):
                f[i+1][j+1]=f[i][j]
            else:
                f[i+1][j+1]=min(f[i][j],f[i+1][j],f[i][j+1])+1
    
    return f[n1][n2]

def check_keyword(keyword,text,percent=0.3):
    keyword=clean_str(keyword)
    text=clean_str(text)
    n_keyword=len(keyword)
    n_text=len(text)
    similar=compare_str(keyword,text[:min(len(text),n_keyword)])
    return similar<=int(percent*min(n_keyword,n_text)),similar
    

def check_line(ct1,ct2,thresh_dis=30): #center 
    dis=abs(ct1[1]-ct2[1])
    return dis<thresh_dis

In [4]:
# list_text after remove noise:
import pickle
import cv2

img = cv2.imread("data/image.jpg")
print(img.shape)
(dt_boxes,list_texts)=pickle.load(open("res.pickle","rb+"))
n=len(dt_boxes)

(3313, 2226, 3)


In [5]:
#get center and draw
def get_center(img,dt_boxes,draw=True):
    image=img.copy()
    color = (0, 255, 0)
    #y,x
    list_centers=[]
    for boxes in dt_boxes:
        y_center=int(sum(boxes[:,0])/4)
        x_center=int(sum(boxes[:,1])/4)
        list_centers.append((y_center,x_center))
        if(draw):
        
            image = cv2.line(image, (int(boxes[0][0]),int(boxes[0][1])), (int(boxes[1][0]),int(boxes[1][1])), color, 1) 
            image = cv2.line(image, (int(boxes[1][0]),int(boxes[1][1])), (int(boxes[2][0]),int(boxes[2][1])), color, 1) 
            image = cv2.line(image, (int(boxes[2][0]),int(boxes[2][1])), (int(boxes[3][0]),int(boxes[3][1])), color, 1)
            image = cv2.line(image, (int(boxes[3][0]),int(boxes[3][1])), (int(boxes[0][0]),int(boxes[0][1])), color, 1)
            image = cv2.circle(image,(y_center,x_center),1,color,1)
    if(draw):
        cv2.imwrite("output/image_with_boxes.jpg",image)
    return list_centers
list_centers=get_center(img,dt_boxes)

In [6]:
list_centers[:2] #(y,x)

[(647, 235), (1541, 233)]

In [7]:
# get_lines and sort
def get_lines(first,last,list_centers): # output list of list index which perline
    lines=[[first]]
    for i in range(first+1,last):

        if(check_line(list_centers[i],list_centers[i-1])):
           lines[-1].append(i)
        else:
           lines.append([i])
    return lines

lines=get_lines(0,n,list_centers)
lines[0:5]

[[0, 1], [2, 3], [4], [5], [6]]

In [8]:
def sort_x(x):
  return list_centers[x][0]
def sort(center_ids):
    center_ids.sort(key=sort_x)
    return center_ids
ids=[]
for x in lines:
    for y in sort(x):
        ids.append(y)

In [9]:
list_texts_new=[]
list_boxes_new=[]
list_centers_new=[]
for i in ids:
    list_texts_new.append(list_texts[i])
    list_boxes_new.append(dt_boxes[i])
    list_centers_new.append(list_centers_new)

In [10]:

Keywords0

['_Header',
 '1.Tên công ty',
 '2.Địa chỉ trụ sở chính',
 '3.Vốn điều lệ',
 '4.Người đại diện theo pháp luật của công ty',
 '_End']

In [11]:
list_texts_new[10]
check_keyword(list_texts_new[10],Keywords0[1])

(True, 1.0)

In [12]:

begin = 0 

indexs={"_Header":0}


def get_index_keyword(keyword,list_text,begin,end):
    similar_min=1000
    index_final=-1
    for i in range(begin,end):
        check,similar=check_keyword(keyword,list_text[i])
        if check and similar < similar_min:
            similar_min=similar
            index_final=i
    return index_final

for keyword in Keywords0[1:-1]:
    index=get_index_keyword(keyword,list_texts_new,0,len(list_texts_new))
    if(index == -1 ):
        print(keyword,"  ",-1)
    else:
        print(keyword,"   :  ",index,list_texts_new[index])
    indexs[keyword]=index
    
if(Contents["_End"][0]!="None" and indexs[Keywords0[-2]]!=-1 ):
    index=get_index_keyword(Contents["_End"][0],list_texts_new,indexs[Keywords0[-2]]+1,len(list_texts_new))
    indexs["_End"]=index
else:
    indexs["_End"]=n

indexs
    

1.Tên công ty    :   10 1. Tên công ty
2.Địa chỉ trụ sở chính    :   15 2. Địa chỉ trụ sở chính
3.Vốn điều lệ    :   22 3. Vốn điều lệ
4.Người đại diện theo pháp luật của công ty    :   29 4. Người đại diện theo pháp luật của công ty


{'_Header': 0,
 '1.Tên công ty': 10,
 '2.Địa chỉ trụ sở chính': 15,
 '3.Vốn điều lệ': 22,
 '4.Người đại diện theo pháp luật của công ty': 29,
 '_End': 48}

In [14]:
# get infor
indexs_final={}
for id,keyword0 in enumerate(Keywords0[:-1]):
    
    print("-------------------------",keyword0,"--------------------------------")
    begin=indexs[keyword0]
    if(begin!=-1):
        end=indexs[Keywords0[id+1]]
        indexs1={}
        keywords1=Contents[keyword0]
        for keyword in keywords1[:-1]:
            if(keyword[0]!="_"):
                index=get_index_keyword(keyword,list_texts_new,begin,end)
                if(index!=-1):
                    begin = index

    #                 print(keyword,"   ",list_texts_new[index])
                indexs1[keyword]=index
            else:
                indexs1[keyword]=begin
        indexs_final[keyword0]=indexs1
    
    print(indexs1)
     

------------------------- _Header --------------------------------
{'Mã Số Doanh Nghiệp': 7, 'Đăng ký lần đầu': 8, 'Đăng ký thay đổi lần thứ': 9}
------------------------- 1.Tên công ty --------------------------------
{'Tên Công ty viết bằng tiếng Việt': 11, 'Tên công ty viết bằng tiếng nước ngoài': 12, 'Tên công ty viết tắt': 14}
------------------------- 2.Địa chỉ trụ sở chính --------------------------------
{'_Địa chỉ': 15, 'Điện thoại': 18, 'Fax': 19, 'Email': 20, 'Website': 21}
------------------------- 3.Vốn điều lệ --------------------------------
{'Vốn điều lệ': 23, 'Bằng Chữ': 25, 'Mệnh giá cổ phần': 26, 'Tổng số cổ phần': 27}
------------------------- 4.Người đại diện theo pháp luật của công ty --------------------------------
{'Họ và tên': 30, 'Giới tính': 31, 'Chức danh': 32, 'Sinh ngày': 33, 'Dân tộc': 35, 'Quốc tịch': 36, 'Loại giấy tờ chứng thực cá nhân': 37, 'Ngày cấp': 41, 'Nơi cấp': 43, 'Nơi đăng kí hộ khẩu thường trú': 44, 'Chỗ ở hiện tại': 46}


In [15]:
last=indexs["_End"]
for i in range(len(Keywords0)-2,-1,-1):
    indexs_final[Keywords0[i]]["_end"]=last
    if(indexs[Keywords0[i]]!=-1):
        last=indexs[Keywords0[i]]
    

In [16]:
indexs_final

{'_Header': {'Mã Số Doanh Nghiệp': 7,
  'Đăng ký lần đầu': 8,
  'Đăng ký thay đổi lần thứ': 9,
  '_end': 10},
 '1.Tên công ty': {'Tên Công ty viết bằng tiếng Việt': 11,
  'Tên công ty viết bằng tiếng nước ngoài': 12,
  'Tên công ty viết tắt': 14,
  '_end': 15},
 '2.Địa chỉ trụ sở chính': {'_Địa chỉ': 15,
  'Điện thoại': 18,
  'Fax': 19,
  'Email': 20,
  'Website': 21,
  '_end': 22},
 '3.Vốn điều lệ': {'Vốn điều lệ': 23,
  'Bằng Chữ': 25,
  'Mệnh giá cổ phần': 26,
  'Tổng số cổ phần': 27,
  '_end': 29},
 '4.Người đại diện theo pháp luật của công ty': {'Họ và tên': 30,
  'Giới tính': 31,
  'Chức danh': 32,
  'Sinh ngày': 33,
  'Dân tộc': 35,
  'Quốc tịch': 36,
  'Loại giấy tờ chứng thực cá nhân': 37,
  'Ngày cấp': 41,
  'Nơi cấp': 43,
  'Nơi đăng kí hộ khẩu thường trú': 44,
  'Chỗ ở hiện tại': 46,
  '_end': 48}}

In [17]:

for title in list(indexs_final.keys()):
    print("________________"+title+"________________")
    print()
    if(indexs[title]!=-1):
        index_title=indexs_final[title]
        keywords_title=list(index_title)
    
        for i in range(len(keywords_title)-1):
            if(index_title[keywords_title[i]]!=-1):
                begin= index_title[keywords_title[i]]
                end=index_title[keywords_title[i+1]]
                print("=============",keywords_title[i],"==============")
                for k in range(begin,end):
                    print(list_texts_new[k])
    print()
    print()
            
    
    

_________________Header________________

============= Mã Số Doanh Nghiệp ==============
Mã số doanh nghiệp: 5700393958
============= Đăng ký lần đầu ==============
Đăng ký lần đầu: ngày 08 tháng 02 năm 2007
============= Đăng ký thay đổi lần thứ ==============
Đăng ký thay đổi lần thứ: 18, ngày 22 tháng 05 năm 2020


________________1.Tên công ty________________

============= Tên Công ty viết bằng tiếng Việt ==============
Tên công ty viết bằng tiếng Việt: CôNG TY Cổ PHÀN TẬP ĐOÀN HOÀNG HÀ
============= Tên công ty viết bằng tiếng nước ngoài ==============
Tên công ty viết bằng tiếng nước ngoài: HOANG HA CORPORATION JOINT
STOCK COMPANY
============= Tên công ty viết tắt ==============
Tên công ty viết tắt: HOANG HA GROUP


________________2.Địa chỉ trụ sở chính________________

============= _Địa chỉ ==============
2. Địa chỉ trụ sở chính
Cụm công nghiệp Kim Sơn, Phường Kim Sơn, Thị xã Đông Triều, Tỉnh Quảng Ninh,
Việt Nam
============= Điện thoại ==============
Điện thoại: 02033.677